In [1]:
from SDP_interaction_inference import simulation
from SDP_interaction_inference.constraints import Constraint
from SDP_interaction_inference import optimization
import numpy as np
import matplotlib.pyplot as plt
from gurobipy import GRB

In [2]:
rng = np.random.default_rng(5)

# Simulation

In [15]:
# simulate
dataset = simulation.simulate_dataset_range_TE([0], rate=5, gene=0.9, tqdm_disable=False)

100%|██████████| 1/1 [00:03<00:00,  3.93s/it]


In [16]:
# downsample
m = 0.5
b = (1 / m) - 1
if b == 0   :
    beta = np.ones(1000)
else:
    beta = rng.beta(1, b, size=1000)
dataset = dataset.downsample(beta)

# Bootstrap

In [17]:
# settings
d = 10

# bootstrap
dataset.bootstrap(d, tqdm_disable=False)

100%|██████████| 1/1 [00:00<00:00,  1.70it/s]


# Model free

In [18]:
#settings
d = 6
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    factorization=True
)

# test feasibility
opt_MF = optimization.ModelFreeOptimization(dataset, d, constraints=constraints, printing=False, silent=True)
opt_MF.analyse_dataset()

opt_MF.result_dict

100%|██████████| 1/1 [00:00<00:00,  6.94it/s]


{0: {'status': 'OPTIMAL', 'time': 0.0029997825622558594, 'cuts': 0}}

# Birth-Death

In [19]:
#settings
d = 4
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True,
    factorization=False
)
fixed = [(1, 1)]#, (3, 1)]

# test feasibility
opt = optimization.BirthDeathOptimization(dataset, d, constraints=constraints, silent=True, fixed=fixed)
opt.analyse_dataset()

opt.result_dict

100%|██████████| 1/1 [00:00<00:00,  9.15it/s]


{0: {'status': 'OPTIMAL', 'time': 0.0019998550415039062, 'cuts': 0}}

# Telegraph

In [34]:
#settings
d = 6
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True,
    telegraph_moments=True,
    telegraph_moments_ineq=True,
    telegraph_factorization=True
)
fixed = [(3, 1), (7, 1), (8, 0)]

# test feasibility
opt_TE = optimization.TelegraphOptimization(dataset, d, constraints=constraints, write_model=True, silent=True, fixed=fixed)
opt_TE.analyse_dataset()

opt_TE.result_dict

  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\willi\Documents\ProjectPaper\M5R_venv\lib\site-packages\SDP_interaction_inference\optimization_utils.py:520: RuntimeWarning: Chained matrix multiplications of MVars is inefficient, collect numeric terms first when building expressions
  model.addConstr(v.T @ variables[f'M_{s}'] @ v >= 0, name=f"Cut_{s}")
100%|██████████| 1/1 [01:47<00:00, 107.94s/it]


{0: {'status': 'OPTIMAL', 'time': 106.15000009536743, 'cuts': 18}}

# General model

In [ ]:
#settings
reactions = [
    "1",
    "xs[0]",
    "1",
    "xs[1]",
    "xs[0] * xs[1]"
]
vrs = [
    [1, 0],
    [-1, 0],
    [0, 1],
    [0, -1],
    [-1, -1]
]
db = 2
R = 5
S = 2
U = []
constraints = Constraint(
    moment_bounds=True,
    moment_matrices=True,
    moment_equations=True
)
fixed = [(1, 1)]
d = 6

# test feasibility
opt_G = optimization.Optimization(dataset, constraints, reactions, vrs, db, R, S, U, d, fixed, printing=False, silent=False)
opt_G.analyse_dataset()

opt_G.result_dict

  0%|          | 0/1 [00:00<?, ?it/s]

Set parameter Username
Set parameter LicenseID to value 2616229
Academic license - for non-commercial use only - expires 2026-01-29
Set parameter TimeLimit to value 300
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300

Optimize a model with 58 rows, 33 columns and 228 nonzeros
Model fingerprint: 0xe83e2e6d
Model has 21 quadratic constraints
Coefficient statistics:
  Matrix range     [1e-04, 1e+00]
  QMatrix range    [1e+00, 1e+01]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [1e-01, 4e+03]
Presolve removed 36 rows and 4 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 14 rows and 4 columns
Presolve time: 0.00s
Presolved: 299 rows, 88 columns, 1040 nonzeros
Presolved model has 5

100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


{0: {'status': 'OPTIMAL', 'time': 0.25999975204467773, 'cuts': 6}}